In [5]:
conda install -c conda-forge glpk

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda





Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\emok\Anaconda3

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    glpk-5.0                   |       h8ffe710_0         3.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  glpk               conda-forge/win-64::glpk-5.0-h8ffe710_0 None




glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    | 7          |   7% 
glpk-5.0             | 3.3 MB    | #2         |  12% 
glpk-5.0             | 3.3 MB    | #9         |  20% 
glpk-5.0             | 3.3 MB    | ##5        |  25% 
glpk-5.0             | 3.3 MB    | ###2       |  33% 
glp

In [28]:
conda install -c conda-forge cbc

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - cbc

Current channels:

  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [29]:
pip install daepy

     -------------------------------------- 129.3/129.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for daepy
Failed to build daepy
  Running setup.py install for daepy: started
  Running setup.py install for daepy: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\daepy
      copying daepy\bvp.py -> build\lib.win-amd64-cpython-39\daepy
      copying daepy\collocation.py -> build\lib.win-amd64-cpython-39\daepy
      copying daepy\continuation.py -> build\lib.win-amd64-cpython-39\daepy
      copying daepy\dae.py -> build\lib.win-amd64-cpython-39\daepy
      copying daepy\derivatives.py -> build\lib.win-amd64-cpython-39\daepy
      copying daepy\nonlinear.py -> build\lib.win-amd64-cpython-39\daepy
      copying daepy\__init__.py -> build\lib.win-amd64-cpython-39\daepy
      creating build\lib.win-amd64-cpython-39\daepy\cheby
      copying daepy\cheby\setup.py -> build\lib.win-amd64-cpython-39\daepy\cheby
      copying daepy\cheby

In [4]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.core import Constraint
from pyomo.dae import *

Static model, when alpha = 0

In [95]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdata.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.C = Var(model.hours, domain=NonNegativeReals)

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())

# Define the constraint
model.d = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of == data['d'][h] + model.C[h])

# Define the objective function
model.Z = Objective(expr=model.k_ipv * model.N_pv +
                   model.k_ion * model.N_on +
                   model.k_iof * model.N_of)

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())


N_pv = 32315.4468498914
N_on = 352866.485014813
N_of = 517826.2190146
Z = 35698618.499111645


Dynamic model with investment costs

In [27]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd_p.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the parameters
model.k_ipv = Param(default=900600)#40)
model.k_ion = Param(default=1700000) #30)
model.k_iof = Param(default=4200000) #46)
model.k_vv2g = Param(default=226.56)   #226.56)
model.eta_charge = Param (default=0.924)
model.eta_discharge = Param(default=0.903)
model.E_s = Param(default=160425)
model.alpha = Param(default=.82)

#Define data
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())
#model.k_vv2g = Param(model.hours, initialize=data['k_vv2g'].to_dict())

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.V = Var(model.hours, domain=NonNegativeReals)
model.V_charge = Var(model.hours, domain=NonNegativeReals)
model.V_discharge = Var(model.hours, domain=NonNegativeReals)
model.C = Var(model.hours, domain=NonNegativeReals)

# Market balance constraints
model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h] + model.C[h])

#State of charge constraint
def s1_constraint(model,h):
    if h == model.hours.first():
        return model.V[h] == model.alpha*model.E_s / 2
    else: 
        return model.V[h] == model.V[h-1] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - model.alpha*data['d_ev'][h]
model.s1 = Constraint(model.hours, rule = s1_constraint)

"""No free lunch"""
model.s2 = Constraint(rule=lambda model:  model.V[model.hours.first()] == model.V[model.hours.last()])

"""Maximum energy storage must be less or equal to capacity"""
model.s3 = Constraint(model.hours, rule=lambda model, h: model.V[h] <= model.alpha*model.E_s)

# Charging and discharging constraints
""" Maximum discharge rate within a single hour """
model.disc1 = Constraint(model.hours, rule=lambda model, h: model.V_discharge[h] <= model.alpha*data['pv2g'][h])

""" Maximum charge rate within a single hour """
model.char1 = Constraint(model.hours, rule=lambda model, h: model.V_charge[h] <= model.alpha*data['pv2g'][h])

# Define objective function
def obj_expression(model):
    return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

model.Z = Objective(expr=obj_expression(model), sense=minimize)

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())

# Access V_discharge and C
V_values = [model.V[h]() for h in model.hours]
V_charge_values = [model.V_charge[h]() for h in model.hours]
V_discharge_values = [model.V_discharge[h]() for h in model.hours]
C_values = [model.C[h]() for h in model.hours]
G_off_values = [data['CF_of'][h]*model.N_of() for h in model.hours]
G_on_values = [data['CF_on'][h]*model.N_on() for h in model.hours]
G_pv_values = [data['CF_pv'][h]*model.N_pv() for h in model.hours]
d_values = [data['d'][h] for h in model.hours]

print("V =", V_values)
print("V_charge=", V_charge_values)
print("V_discharge=", V_discharge_values)
print("C =", C_values)
print("G_off =", G_off_values)
print("G_on =", G_on_values)
print("G_pv =", G_pv_values)
print("d =", d_values)

N_pv = 0.0
N_on = 27638.6115112432
N_of = 2301.85034332288
Z = 58045802157.29269
V = [65774.25, 65504.6513247203, 65281.7374608444, 65148.1602386863, 65067.2561937927, 65003.461765644, 64933.1901254419, 64861.5741582099, 64802.4237688919, 63622.462501745, 61495.050474148, 59270.6134527314, 57736.9534155515, 56863.8757843696, 56379.5514189418, 55916.0030167043, 55516.9823119045, 55198.3768058055, 54955.6646711249, 54752.060231861, 54566.6653132756, 54398.1355883388, 54249.2819226585, 54141.0024909731, 54081.729890107, 54044.4553679129, 54017.2021926693, 53994.7152678046, 53962.5736306339, 53907.2117993094, 53807.6093875447, 53690.1640897461, 53541.432635614, 53276.1113645204, 52890.7783531165, 52377.8564854153, 51775.7201875443, 51101.3568647005, 50435.1817155845, 49800.5371458983, 49232.9867161617, 48754.7729281428, 48392.1712646999, 48085.7869134188, 47818.3880460062, 47595.1075474858, 47424.7446493263, 47323.1868527907, 47265.0141558582, 47229.4505953385, 47194.2536694633, 47150.1353

In [ ]:
# List of alpha values
alpha_values = np.arange(0, 1.01, 0.01)

# Arrays to store results
N_pv_opt_array = np.zeros(len(alpha_values))
N_on_opt_array = np.zeros(len(alpha_values))
N_of_opt_array = np.zeros(len(alpha_values))
Z_array = np.zeros(len(alpha_values))
d_sum_array = np.zeros(len(alpha_values))
C_sum_array = np.zeros(len(alpha_values))
V_discharge_sum_array = np.zeros(len(alpha_values))

# Loop over each alpha value
for i, alpha in enumerate(alpha_values):

    # Load data into a Pandas DataFrame
    data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd_p.csv')

    # Create a Pyomo model
    model = ConcreteModel()

    # Define the sets
    model.hours = Set(initialize=data.index, doc='Set of hours')

    # Define the parameters
    model.k_ipv = Param(default=900600)#40)
    model.k_ion = Param(default=1700000) #30)
    model.k_iof = Param(default=4200000) #46)
    model.k_vv2g = Param(default=226.56)   #226.56)
    model.eta_charge = Param (default=0.924)
    model.eta_discharge = Param(default=0.903)
    model.E_s = Param(default=160425)
    model.alpha = Param(default=.82)

    #Define data
    model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
    model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
    model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
    model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
    model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())
    #model.k_vv2g = Param(model.hours, initialize=data['k_vv2g'].to_dict())

    # Define the variables
    model.N_pv = Var(domain=NonNegativeReals, initialize=0)
    model.N_on = Var(domain=NonNegativeReals, initialize=0)
    model.N_of = Var(domain=NonNegativeReals, initialize=0)
    model.V = Var(model.hours, domain=NonNegativeReals)
    model.V_charge = Var(model.hours, domain=NonNegativeReals)
    model.V_discharge = Var(model.hours, domain=NonNegativeReals)
    model.C = Var(model.hours, domain=NonNegativeReals)

    # Market balance constraints
    model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                    model.CF_on[h] * model.N_on +
                                                    model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h] + model.C[h])

    #State of charge constraint
    def s1_constraint(model,h):
        if h == model.hours.first():
            return model.V[h] == model.alpha*model.E_s / 2
        else: 
            return model.V[h] == model.V[h-1] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - model.alpha*data['d_ev'][h]
    model.s1 = Constraint(model.hours, rule = s1_constraint)

    """No free lunch"""
    model.s2 = Constraint(rule=lambda model:  model.V[model.hours.first()] == model.V[model.hours.last()])

    """Maximum energy storage must be less or equal to capacity"""
    model.s3 = Constraint(model.hours, rule=lambda model, h: model.V[h] <= model.alpha*model.E_s)

    # Charging and discharging constraints
    """ Maximum discharge rate within a single hour """
    model.disc1 = Constraint(model.hours, rule=lambda model, h: model.V_discharge[h] <= model.alpha*data['pv2g'][h])

    """ Maximum charge rate within a single hour """
    model.char1 = Constraint(model.hours, rule=lambda model, h: model.V_charge[h] <= model.alpha*data['pv2g'][h])

    # Define objective function
    def obj_expression(model):
        return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

    model.Z = Objective(expr=obj_expression(model), sense=minimize)

    # Solve the optimization problem
    solver = SolverFactory('glpk')
    solver.solve(model)


    # Retrieve the optimal values of decision variables
    N_pv_opt = model.N_pv()
    N_on_opt = model.N_on()
    N_of_opt = model.N_of()
    Z_opt = model.Z()
    sum_C = [sum(model.C[h]() for h in model.hours)][0]
    sum_d = [sum(data['d'][h] for h in model.hours)][0]
    sum_V_discharge = [sum(model.V_discharge[h]() for h in model.hours)][0]

    # Store results in arrays
    N_pv_opt_array[i] = N_pv_opt
    N_on_opt_array[i] = N_on_opt
    N_of_opt_array[i] = N_of_opt
    Z_array[i] = Z_opt
    d_sum_array[i] = sum_d
    C_sum_array[i] = sum_C
    V_discharge_sum_array[i] = sum_V_discharge

    print([i])




In [ ]:


# Create a dataframe from the arrays
results_df = pd.DataFrame({
    'alpha': alpha_values,
    'N_pv': N_pv_opt_array,
    'N_on': N_on_opt_array,
    'N_of': N_of_opt_array,
    'Z': Z_array,
    'sum_d': d_sum_array,
    'sum_C': C_sum_array,
    'sum_V_discharge': V_discharge_sum_array
})

# Print the results
print(results_df)

FIGURES

In [28]:
sum_C = [sum(model.C[h]() for h in model.hours)][0]
sum_d = [sum(data['d'][h] for h in model.hours)][0]

share_C = sum_C/(sum_d+sum_C)

print(share_C)

0.5246639798543965


In [ ]:
alpha = 0, 0.25, 0.5, 0.75, 1
share_curtailment = 0.9871059090755362 , 0.9729879101983462, 0.9191847383648472, 0.741110796380353, 0 

OLD MODELS

In [ ]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.k_vv2g = Param(default=226.56)
model.eta_charge = Param (default=0.924)
model.eta_discharge = Param(default=0.903)
model.E_s = Param(default=160425)
model.alpha = Param(default=0.25)

#Define data
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.V = Var(model.hours, domain=NonNegativeReals)
model.V_charge = Var(model.hours, domain=NonNegativeReals)
model.V_discharge = Var(model.hours, domain=NonNegativeReals)
model.C = Var(model.hours, domain=NonNegativeReals)

# Market balance constraints
model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h] + model.C[h])

#State of charge constraint
def s1_constraint(model,h):
    if h == model.hours.first():
        return model.V[h] == model.alpha*model.E_s / 2
    else: 
        return model.V[h] == model.V[h-1] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h]
model.s1 = Constraint(model.hours, rule = s1_constraint)

"""No free lunch"""
model.s2 = Constraint(rule=lambda model:  model.V[model.hours.first()] == model.V[model.hours.last()])

"""Maximum energy storage must be less or equal to capacity"""
model.s3 = Constraint(model.hours, rule=lambda model, h: model.V[h] <= model.alpha*model.E_s)

# Charging and discharging constraints
""" Maximum discharge rate within a single hour """
model.disc1 = Constraint(model.hours, rule=lambda model, h: model.V_discharge[h] <= model.alpha*data['pv2g'][h])

""" Maximum charge rate within a single hour """
model.char1 = Constraint(model.hours, rule=lambda model, h: model.V_charge[h] <= model.alpha*data['pv2g'][h])

# Define objective function
def obj_expression(model):
    return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

model.Z = Objective(expr=obj_expression(model), sense=minimize) # Use the defined objective expression with the 'h' variable

# Solve the optimization problem
solver = SolverFactory('glpk')
#solver.solve(model)
results = solver.solve(model,tee=True)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())